# Композиции алгоритмов регрессии

## Необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

## Подготовка данных

In [4]:
path_to_file = '../datasets/regression/metro_interstate_traffic_volume_preprocessed.csv'
dataset = pd.read_csv(path_to_file)

In [5]:
dataset.head()

,Unnamed: 0,temp,rain_1h,snow_1h,clouds_all,date_time,traffic_volume,h_Christmas Day,h_Columbus Day,h_Independence Day,...,wd_sleet,wd_smoke,wd_snow,wd_thunderstorm,wd_thunderstorm with drizzle,wd_thunderstorm with heavy rain,wd_thunderstorm with light drizzle,wd_thunderstorm with light rain,wd_thunderstorm with rain,wd_very heavy rain
0,0,288.28,0.0,0.0,40,2012-10-02 09:00:00,5545,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,289.36,0.0,0.0,75,2012-10-02 10:00:00,4516,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,289.58,0.0,0.0,90,2012-10-02 11:00:00,4767,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,290.13,0.0,0.0,90,2012-10-02 12:00:00,5026,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,291.14,0.0,0.0,75,2012-10-02 13:00:00,4918,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Удалим ненужные столбцы, разобьём выборку и отмасштабируем

In [6]:
dataset = dataset.drop(['Unnamed: 0', 'date_time'], axis=1)

In [7]:
X = dataset.iloc[:, [0, 1, 2, 3] + list(range(5, 65))]
y = dataset.iloc[:, 4]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Бэггинг

**Бэггинг**: каждый базовый аллгоритм обучается на подвыборке исходной обучающей выборки. Формирование подвыборке происходит при помощи *бутстрапа* (выбор с возвращением элементов из исходного множества)

## Стэкинг

**Стэкинг**: на основе обученных базовых алгоритмов строится новый *метаалгоритм*, для вывода предсказаний, основанных на множественных предсказаниях, возвращаемых этими базовыми алгоритмами

## Градиентный бустинг

**Градиентный бустинг**: градиентный бустинг сводит задачу к градиентному спуску: на каждой итерации мы подгоняем слабого ученика к антиградиенту текущей ошибки подбора по отношению к текущей модели ансамбля